Install amplpy, pandas and other packages.

In [ ]:
!pip install -q amplpy ampltools pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 14.3 MB/s eta 0:00:00


Setup AMPL and select solvers.

In [ ]:
# Google Colab & AMPL integration

MODULES, LICENSE_UUID = ["coin", 'gurobi', "cplex", "highs", "gokestrel"], "6b31af08-ff1f-429f-ad0b-4bb913b68f75"

from amplpy import tools
from ampltools import cloud_platform_name, ampl_notebook, register_magics

# instantiate AMPL object and register magics
if cloud_platform_name() is None:
    ampl = AMPL() # Use local installation of AMPL
else:
    ampl = tools.ampl_notebook(modules=MODULES, license_uuid=LICENSE_UUID, g=globals())

register_magics(ampl_object=ampl)

Licensed to Bundle #6300.6669 expiring 20231231: INFO 645 Prescriptive Analytics, Prof. Paul Brooks, Virginia Commonwealth University.


In [ ]:
# Now set the data in AMPL
projects = [1, 2, 3, 4, 5, 6]
returns = {1: 32.40, 2: 35.80, 3: 17.75, 4: 14.80, 5: 18.20, 6: 12.35}
cash_outlays = {
    (1, 1): 10.50, (1, 2): 14.40, (1, 3): 2.20, (1, 4): 2.40,
    (2, 1): 8.30,  (2, 2): 12.60, (2, 3): 9.50, (2, 4): 3.10,
    (3, 1): 10.20,  (3, 2): 14.20, (3, 3): 5.60, (3, 4): 4.20,
    (4, 1): 7.20,  (4, 2): 10.50, (4, 3): 7.50, (4, 4): 5.00,
    (5, 1): 12.30,  (5, 2): 10.10, (5, 3): 8.30, (5, 4): 6.30,
    (6, 1): 9.20,  (6, 2): 7.80, (6, 3): 6.90, (6, 4): 5.10,
}
yearly_funds = {1: 30, 2: 30, 3: 35, 4: 20}

In [ ]:
ampl.eval ('''

reset;

set PROJECTS;  # Set of projects
set YEARS;     # Set of years

param Returns {i in PROJECTS};           # Expected returns for each project
param CashOutlays {i in PROJECTS, j in YEARS}; # Cash outlays for each project in each year
param YearlyFunds {i in YEARS};          # Available funds in each year

# Decision variable: fraction of each project to undertake
var x {i in PROJECTS} >= 0, <= 1;

# Objective: Maximize total expected returns
maximize TotalReturns:
    sum {i in PROJECTS} Returns[i] * x[i];

# Yearly budget constraints
subject to Budget_Constraint {j in YEARS}:
    sum {i in PROJECTS} CashOutlays[i, j] * x[i] <= YearlyFunds[j];
''')

In [ ]:
ampl.set['PROJECTS'] = projects
ampl.set['YEARS'] = list(yearly_funds.keys())
ampl.param['Returns'] = returns
ampl.param['CashOutlays'] = cash_outlays
ampl.param['YearlyFunds'] = yearly_funds

In [ ]:
ampl.setOption('solver', 'cbc')
ampl.solve()

cbc 2.10.10: cbc 2.10.10: optimal solution; objective 73.60594059
0 simplex iterations
 


In [ ]:
ampl.eval('''expand;''')

maximize TotalReturns:
	32.4*x[1] + 35.8*x[2] + 17.75*x[3] + 14.8*x[4] + 18.2*x[5] + 12.35*x[6];

subject to Budget_Constraint[1]:
	10.5*x[1] + 8.3*x[2] + 10.2*x[3] + 7.2*x[4] + 12.3*x[5] + 9.2*x[6] <= 
	30;

subject to Budget_Constraint[2]:
	14.4*x[1] + 12.6*x[2] + 14.2*x[3] + 10.5*x[4] + 10.1*x[5] + 7.8*x[6]
	 <= 30;

subject to Budget_Constraint[3]:
	2.2*x[1] + 9.5*x[2] + 5.6*x[3] + 7.5*x[4] + 8.3*x[5] + 6.9*x[6] <= 35;

subject to Budget_Constraint[4]:
	2.4*x[1] + 3.1*x[2] + 4.2*x[3] + 5*x[4] + 6.3*x[5] + 5.1*x[6] <= 20;



In [ ]:
# Print results
obj = ampl.get_objective('TotalReturns')
print("\n")
print("Total_NPV is: ", obj.get().value(), "\n")
print("Projects choosen:")
ampl.display('x');



Total_NPV is:  73.60594059405939 

Projects choosen:
x [*] :=
1  1
2  1
3  0
4  0
5  0.29703
6  0
;

